# Chart generations Notebook

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot
from pathlib import Path

In [2]:
offline.init_notebook_mode(connected=True)

## Bubble Chart for Emissions Intensity


In [3]:
company_id = 1

# Step 1: Set up the data sources

In [4]:
path = Path(os.path.dirname (os.getcwd()))
XLSX_PATH = os.path.join(path.parent.parent, 'sp100.xlsx')

COLS_TO_USE = {
    'companies':['company_id', 'company_name'],
    'ghg_quant':['company_id', 'ghg_scope_1','ghg_loc_scope_2','ghg_mkt_scope_2', 'reporting_year', "Source"],
    'grouping':['company_id', 'Sector1'],
    'financials':['company_id', 'Revenue_num_tradingview']
}


# Step 2: Merge the tables

In [5]:
all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))

In [6]:
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )

In [7]:
merged_df

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
0,1.0,3M,CDP,2020.0,NR,NR,NR,Industrials,3.218400e+10
1,1.0,3M,Public,2020.0,3600000,1680000,1250000,Industrials,3.218400e+10
2,1.0,3M,Final,2020.0,3600000,1680000,1250000,Industrials,3.218400e+10
3,1.0,3M,CDP,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
4,1.0,3M,Public,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
...,...,...,...,...,...,...,...,...,...
1351,113.0,Linde plc,Public,2018.0,16872000,NR,22333000,Materials,2.724300e+10
1352,113.0,Linde plc,Final,2018.0,16872000,0,22333000,Materials,2.724300e+10
1353,113.0,Linde plc,CDP,2017.0,NR,NR,NR,Materials,2.724300e+10
1354,113.0,Linde plc,Public,2017.0,NR,NR,NR,Materials,2.724300e+10


In [8]:
# Select Final Figures and Year 2019
cond1 = (merged_df['reporting_year'] == 2019)
cond2 = (merged_df['Source']== 'Final')
filter_cond = cond1 & cond2
merged_df = merged_df.loc[filter_cond]

In [9]:
merged_df

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
5,1.0,3M,Final,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
17,2.0,Abbott Laboratories,Final,2019.0,533000,518000,439000,Healthcare,3.460800e+10
29,3.0,AbbVie Inc.,Final,2019.0,314421,308204,249777,Healthcare,4.580400e+10
41,4.0,Accenture,Final,2019.0,18923,281489,214680,Technology,4.432700e+10
53,5.0,Adobe Inc.,Final,2019.0,11816,56113,43526,Technology,1.286500e+10
...,...,...,...,...,...,...,...,...,...
1301,109.0,Halliburton Co.,Final,2019.0,4511499,249259,NR,NaN,NaN
1313,110.0,HP Inc (ex Hewlett-Packard),Final,2019.0,57509,313002,177018,NaN,NaN
1325,111.0,Norfolk Southern Corp.,Final,2019.0,4784047,201474,201474,NaN,NaN
1337,112.0,T-Mobile (ex T-Mobile US),Final,2019.0,37175,1393730,729490,Communication Services,6.839700e+10


In [10]:
merged_df.head()

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
5,1.0,3M,Final,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
17,2.0,Abbott Laboratories,Final,2019.0,533000,518000,439000,Healthcare,3.460800e+10
29,3.0,AbbVie Inc.,Final,2019.0,314421,308204,249777,Healthcare,4.580400e+10
41,4.0,Accenture,Final,2019.0,18923,281489,214680,Technology,4.432700e+10
53,5.0,Adobe Inc.,Final,2019.0,11816,56113,43526,Technology,1.286500e+10


# Step 3: Filter the sector

In [11]:
a =  merged_df[merged_df['company_id']==company_id]['Sector1']
if len(a) > 0:
    sector = a.iloc[0]
merged_df_sector = merged_df.loc[merged_df['Sector1']==sector]

In [12]:
merged_df_sector.head()

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
5,1.0,3M,Final,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
233,20.0,Boeing,Final,2019.0,613000,871000,733000,Industrials,5.816100e+10
281,24.0,Caterpillar Inc.,Final,2019.0,905000,926000,869000,Industrials,4.174600e+10
473,40.0,Emerson Electric,Final,2019.0,172553,678912,682177,Industrials,1.679000e+10
521,44.0,FedEx,Final,2019.0,15406173,995988,995988,Industrials,8.379400e+10


# Step 4: Calculate Scope 1 + Scope 2

In [13]:
scope1 = pd.to_numeric(merged_df_sector['ghg_scope_1'], errors = 'coerce')
scope2_loc = pd.to_numeric(merged_df_sector['ghg_loc_scope_2'], errors = 'coerce')
scope2_mkt = pd.to_numeric(merged_df_sector['ghg_mkt_scope_2'], errors = 'coerce')

merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_loc= scope1 + scope2_loc)
merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_mkt= scope1 + scope2_mkt)
merged_df_sector['scope1_plus_scope2'] = merged_df_sector[['scope1_plus_scope2_loc','scope1_plus_scope2_mkt']].min(axis=1)

In [14]:
fields = ['company_id','company_name','ghg_scope_1', 'Sector1','Revenue_num_tradingview','scope1_plus_scope2']
merged_df_sector = merged_df_sector [fields]

In [15]:
merged_df_sector

,company_id,company_name,ghg_scope_1,Sector1,Revenue_num_tradingview,scope1_plus_scope2
5,1.0,3M,4050000,Industrials,3.218400e+10,5370000.0
233,20.0,Boeing,613000,Industrials,5.816100e+10,1346000.0
281,24.0,Caterpillar Inc.,905000,Industrials,4.174600e+10,1774000.0
473,40.0,Emerson Electric,172553,Industrials,1.679000e+10,851465.0
521,44.0,FedEx,15406173,Industrials,8.379400e+10,16402161.0
545,46.0,General Dynamics,317081,Industrials,3.792500e+10,762200.0
557,47.0,General Electric,997410,Industrials,7.989300e+10,2385025.0
617,52.0,Honeywell Inc.,1090649,Industrials,3.264000e+10,2024133.0
701,59.0,Lockheed Martin Corp.,305362,Industrials,6.539800e+10,771435.0
965,81.0,Raytheon Technologies,0,Industrials,5.666900e+10,0.0


# Step 5: Remove companies without valid data

In [16]:
merged_df_sector

,company_id,company_name,ghg_scope_1,Sector1,Revenue_num_tradingview,scope1_plus_scope2
5,1.0,3M,4050000,Industrials,3.218400e+10,5370000.0
233,20.0,Boeing,613000,Industrials,5.816100e+10,1346000.0
281,24.0,Caterpillar Inc.,905000,Industrials,4.174600e+10,1774000.0
473,40.0,Emerson Electric,172553,Industrials,1.679000e+10,851465.0
521,44.0,FedEx,15406173,Industrials,8.379400e+10,16402161.0
545,46.0,General Dynamics,317081,Industrials,3.792500e+10,762200.0
557,47.0,General Electric,997410,Industrials,7.989300e+10,2385025.0
617,52.0,Honeywell Inc.,1090649,Industrials,3.264000e+10,2024133.0
701,59.0,Lockheed Martin Corp.,305362,Industrials,6.539800e+10,771435.0
965,81.0,Raytheon Technologies,0,Industrials,5.666900e+10,0.0


In [17]:
merged_df_sector = merged_df_sector.dropna()

# Step 6 (optional): Calculate GHG Intensity

In [18]:
merged_df_sector = merged_df_sector.assign(intensity=merged_df_sector['scope1_plus_scope2'] / merged_df_sector['Revenue_num_tradingview'])

In [19]:
merged_df_sector

,company_id,company_name,ghg_scope_1,Sector1,Revenue_num_tradingview,scope1_plus_scope2,intensity
5,1.0,3M,4050000,Industrials,3.218400e+10,5370000.0,0.000167
233,20.0,Boeing,613000,Industrials,5.816100e+10,1346000.0,0.000023
281,24.0,Caterpillar Inc.,905000,Industrials,4.174600e+10,1774000.0,0.000042
473,40.0,Emerson Electric,172553,Industrials,1.679000e+10,851465.0,0.000051
521,44.0,FedEx,15406173,Industrials,8.379400e+10,16402161.0,0.000196
545,46.0,General Dynamics,317081,Industrials,3.792500e+10,762200.0,0.000020
557,47.0,General Electric,997410,Industrials,7.989300e+10,2385025.0,0.000030
617,52.0,Honeywell Inc.,1090649,Industrials,3.264000e+10,2024133.0,0.000062
701,59.0,Lockheed Martin Corp.,305362,Industrials,6.539800e+10,771435.0,0.000012
965,81.0,Raytheon Technologies,0,Industrials,5.666900e+10,0.0,0.000000


# Step 7: Prepare the chart by fetching specific data series

In [20]:
if len(merged_df_sector[merged_df_sector['company_id']==company_id]['company_name']) > 0:
    corp_x = merged_df_sector[merged_df_sector['company_id']==company_id]['scope1_plus_scope2'].iloc[0]
    corp_y = merged_df_sector[merged_df_sector['company_id']==company_id]['Revenue_num_tradingview'].iloc[0]
    corp_name = merged_df_sector[merged_df_sector['company_id']==company_id]['company_name'].iloc[0]


x0 = merged_df_sector['scope1_plus_scope2']
y0 = merged_df_sector['Revenue_num_tradingview']
intensity_data = 10000000 * merged_df_sector['intensity']
#index_corp = merged_df_sector.loc[merged_df_sector['company_id'] == company_id].index
#merged_df_sector.drop(index_corp , inplace=True)
x_median_x = [x0.min(), x0.max()]
y_median_x = [y0.median(), y0.median()]
x_median_y = [x0.median(), x0.median()]
y_median_y = [y0.min(), y0.max()]

# Step 7: Produce the chart

In [21]:
layout = go.Layout (
    title = '<b>Operational Emissions Intensity Benchmark</b><br>Sector: ' + sector,
    title_x = 0.5,
    titlefont = dict(
        family = 'Arial',
        size = 16),
    plot_bgcolor = 'antiquewhite',
    xaxis =  dict(
        autorange = "reversed",
        type = 'log',
        title = '<i>high</i>----------<b>Scope1+2 Emissions</b>----------<i>low</i>'),
    yaxis = dict(
        type = 'log',
        title = 'Revenue'),
        )

In [22]:
intensity_data

5       1668.530947
233      231.426557
281      424.950893
473      507.126266
521     1957.438599
545      200.975610
557      298.527405
617      620.138787
701      117.960029
965        0.000000
1097    5314.426867
1121    1771.129430
Name: intensity, dtype: float64

In [26]:
trace1 = go.Scatter(x=x_median_y, 
                    y=y_median_y, 
                    showlegend = False, 
                    name='Median y',
                    mode = "lines",
                    line = dict(color='gray', 
                                width=2, 
                                dash='dash'))

trace2 = go.Scatter(x=x_median_x, 
                         y=y_median_x, 
                         showlegend = False, 
                         name='Median x', 
                         mode = "lines",
                         line = {'color':'gray', 
                                 'width':2, 
                                 'dash':'dash'}
                        )

trace3 = go.Scatter(
    x=merged_df_sector['scope1_plus_scope2'], 
    y=merged_df_sector['Revenue_num_tradingview'],
    showlegend = False,
    name="benchmark", 
    text=merged_df_sector['company_name'],
    mode = 'markers+text',
    line = {'color':'black'}, 
    marker = dict (
        color=intensity_data,
        size=25,
        showscale=True,
        colorscale='Hot_r',
        line=dict(
                color='black',
                width=1,
            )

        ),
    hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
    textposition='top center'
    )

fig = go.Figure(data = [trace1, trace2, trace3], 
                layout = layout)
                
fig.update_layout(
    autosize=True,
    #width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=80,
        pad=4
    ),
    paper_bgcolor="#eeebf0",
    plot_bgcolor = '#eeebf0',
)
config = {'displaylogo': False}
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=[corp_x],
        y=[corp_y],
        text = [corp_name],
        marker=dict(
            color='rgba(255,0,0, 0.01)',#'rgba(135, 206, 250, 0.01)',
            size=25,
            line=dict(
                color='red',
                width=3,
            )
        ),
        
        hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
        showlegend=False
    )
)

In [27]:
offline.iplot(fig, config = config)

In [25]:
    path = Path(os.path.dirname (os.getcwd()))
    XLSX_PATH = os.path.join(path.parent.parent, 'sp100.xlsx')
    
    COLS_TO_USE = {
    'companies':['company_id', 'company_name'],
    'ghg_quant':['company_id', 'ghg_scope_1','ghg_loc_scope_2','ghg_mkt_scope_2', 'reporting_year', "Source"],
    'grouping':['company_id', 'Sector1'],
    'financials':['company_id', 'Revenue_num_tradingview']
    }

    all_df = []
    for sheetname, cols in COLS_TO_USE.items():
        all_df.append(pd.read_excel(
            XLSX_PATH, 
            sheet_name = sheetname,
            engine = 'openpyxl', 
            usecols = cols
            ))
    merged_df = all_df[0]
    for i in range(len(COLS_TO_USE) - 1):
        merged_df = pd.merge(
            left = merged_df,
            right = all_df[i+1],
            how="left",
            on="company_id"
        )
    # Select Final Figures and Year 2019
    cond1 = (merged_df['reporting_year'] == 2019)
    cond2 = (merged_df['Source']== 'Final')
    filter_cond = cond1 & cond2
    merged_df = merged_df.loc[filter_cond]

    a =  merged_df[merged_df['company_id']==company_id]['Sector1']
    if len(a) > 0:
        sector = a.iloc[0]
    merged_df_sector = merged_df.loc[merged_df['Sector1']==sector]

    scope1 = pd.to_numeric(merged_df_sector['ghg_scope_1'], errors = 'coerce')
    scope2_loc = pd.to_numeric(merged_df_sector['ghg_loc_scope_2'], errors = 'coerce')
    scope2_mkt = pd.to_numeric(merged_df_sector['ghg_mkt_scope_2'], errors = 'coerce')

    merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_loc= scope1 + scope2_loc)
    merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_mkt= scope1 + scope2_mkt)
    merged_df_sector['scope1_plus_scope2'] = merged_df_sector[['scope1_plus_scope2_loc','scope1_plus_scope2_mkt']].min(axis=1)

    fields = ['company_id','company_name','ghg_scope_1', 'Sector1','Revenue_num_tradingview','scope1_plus_scope2']
    merged_df_sector = merged_df_sector [fields]

    merged_df_sector = merged_df_sector.dropna()
    merged_df_sector = merged_df_sector.assign(intensity=merged_df_sector['scope1_plus_scope2'] / merged_df_sector['Revenue_num_tradingview'])
    corp_x = merged_df_sector[merged_df_sector['company_id']==company_id]['scope1_plus_scope2'].iloc[0]
    corp_y = merged_df_sector[merged_df_sector['company_id']==company_id]['Revenue_num_tradingview'].iloc[0]
    corp_name = merged_df_sector[merged_df_sector['company_id']==company_id]['company_name'].iloc[0]
    x0 = merged_df_sector['scope1_plus_scope2']
    y0 = merged_df_sector['Revenue_num_tradingview']
    intensity_data = 10000000 * merged_df_sector['intensity']
    #index_corp = merged_df_sector.loc[merged_df_sector['company_id'] == company_id].index
    #merged_df_sector.drop(index_corp , inplace=True)
    x_median_x = [x0.min(), x0.max()]
    y_median_x = [y0.median(), y0.median()]
    x_median_y = [x0.median(), x0.median()]
    y_median_y = [y0.min(), y0.max()]

    layout = go.Layout (
        title = '<b>Operational Emissions Intensity Benchmark</b><br>Sector: ' + sector,
        title_x = 0.5,
        titlefont = dict(
            family = 'Arial',
            size = 16),
        plot_bgcolor = 'antiquewhite',
        xaxis =  dict(
            autorange = "reversed",
            type = 'log',
            title = '<i>high</i>----------<b>Scope1+2 Emissions</b>----------<i>low</i>'),
        yaxis = dict(
            type = 'log',
            title = 'Revenue'),
            )

    trace1 = go.Scatter(x=x_median_y, 
                        y=y_median_y, 
                        showlegend = False, 
                        name='Median y',
                        mode = "lines",
                        line = dict(color='gray', 
                                    width=2, 
                                    dash='dash'))

    trace2 = go.Scatter(x=x_median_x, 
                            y=y_median_x, 
                            showlegend = False, 
                            name='Median x', 
                            mode = "lines",
                            line = {'color':'gray', 
                                    'width':2, 
                                    'dash':'dash'}
                            )

    trace3 = go.Scatter(
        x=merged_df_sector['scope1_plus_scope2'], 
        y=merged_df_sector['Revenue_num_tradingview'],
        showlegend = False,
        name="benchmark", 
        text=merged_df_sector['company_name'],
        mode = 'markers+text',
        line = {'color':'black'}, 
        marker = dict (
            color=intensity_data,
            size=25,
            showscale=True,
            colorscale='Hot_r',
            line=dict(
                    color='black',
                    width=1,
                )

            ),
        hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
        textposition='top center'
        )

    fig = go.Figure(data = [trace1, trace2, trace3], 
                    layout = layout)
                    
    
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=[corp_x],
            y=[corp_y],
            text = [corp_name],
            marker=dict(
                color='rgba(255,0,0, 0.01)',#'rgba(135, 206, 250, 0.01)',
                size=25,
                line=dict(
                    color='red',
                    width=3,
                )
            ),
            
            hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
            showlegend=False
        )
    )

    
    full_fig = fig.full_figure_for_development(warn=False)

    x_range = full_fig.layout.xaxis.range
    y_range = full_fig.layout.yaxis.range
                    
    fig.update_layout(
        autosize=False,
        width=1000,
        height=500,
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=80,
            pad=4
        ),
        paper_bgcolor="#eeebf0",
        plot_bgcolor = '#eeebf0',
    )